In [1]:
# Imports the Google Cloud client library
from google.cloud import language_v1

# Instantiates a client
client = language_v1.LanguageServiceClient()

# The text to analyze
text = '''GENEVA – The remains of a climber discovered in the Swiss Alps in 2022 have been identified as \
    those of a British mountaineer who went missing 52 years ago, local police said on Thursday.\
        It is the latest in a series of discoveries of remains of long-missing climbers revealed as the Alps’ glaciers melt and recede because of global warming.The climber was reported missing in July 1971, \
            but search teams at the time turned up nothing, said police in the canton of Valais, south-west Switzerland.Then on Aug 22, 2022, two climbers found human remains on the Chessjengletscher glacier near Saas-Fee,\
    an Alpine village in the Saas Valley.It took a year to identify the person as experts worked their \
        way through the case files of missing climbers.Finally, with the help of Interpol Manchester and the police in Scotland, a relative was found and a DNA sample allowed them to identify the British mountaineer\
            , \the Swiss police said in a statement.\
        The climber was formally identified on Aug 30.Increasing numbers of human remains, some of them of climbers missing for decades, have been discovered in recent years as glaciers in the Alps melt because of global warming.In late July, \
            the remains of a German climber who went missing in 1986 were discovered on another Swiss glacier. AFP                  ",
'''
document = language_v1.types.Document(
    content=text, type_=language_v1.types.Document.Type.PLAIN_TEXT
)

# Detects the sentiment of the text
entity = client.analyze_entities(
    request={"document": document}
)

print(f"Text: {text}")
print(entity.entities)

Text: GENEVA – The remains of a climber discovered in the Swiss Alps in 2022 have been identified as     those of a British mountaineer who went missing 52 years ago, local police said on Thursday.        It is the latest in a series of discoveries of remains of long-missing climbers revealed as the Alps’ glaciers melt and recede because of global warming.The climber was reported missing in July 1971,             but search teams at the time turned up nothing, said police in the canton of Valais, south-west Switzerland.Then on Aug 22, 2022, two climbers found human remains on the Chessjengletscher glacier near Saas-Fee,    an Alpine village in the Saas Valley.It took a year to identify the person as experts worked their         way through the case files of missing climbers.Finally, with the help of Interpol Manchester and the police in Scotland, a relative was found and a DNA sample allowed them to identify the British mountaineer            , 	he Swiss police said in a statement.    

In [12]:
for i in range(len(entity.entities)):
    print(entity.entities[i])
    if i >1:
        break
    

name: "climber"
type_: PERSON
salience: 0.301220328
mentions {
  text {
    content: "climber"
    begin_offset: -1
  }
  type_: COMMON
}
mentions {
  text {
    content: "climber"
    begin_offset: -1
  }
  type_: COMMON
}
mentions {
  text {
    content: "climber"
    begin_offset: -1
  }
  type_: COMMON
}

name: "mountaineer"
type_: PERSON
salience: 0.166266277
mentions {
  text {
    content: "mountaineer"
    begin_offset: -1
  }
  type_: COMMON
}

name: "remains"
type_: OTHER
salience: 0.0640264302
mentions {
  text {
    content: "remains"
    begin_offset: -1
  }
  type_: COMMON
}



In [2]:
entity.entities[2]

name: "remains"
type_: OTHER
salience: 0.0640264302
mentions {
  text {
    content: "remains"
    begin_offset: -1
  }
  type_: COMMON
}

In [18]:
types = []
for en in entity.entities:
    # if str(en.type_) == 'Type.ORGANIZATION':
    #     print(en)
    if str(en.type_) not in types:
        types.append(str(en.type_))
        # print(str(en.type_))
types


['Type.PERSON',
 'Type.OTHER',
 'Type.LOCATION',
 'Type.ORGANIZATION',
 'Type.EVENT',
 'Type.WORK_OF_ART',
 'Type.CONSUMER_GOOD',
 'Type.DATE',
 'Type.NUMBER']

In [36]:
hashmap = {}
for en in entity.entities:
    type = str(en.type_)[5:].lower()
    if type not in hashmap:
        hashmap[type] = []
    hashmap[type].append(en)
        

In [34]:
hashmap

{'PERSON': [name: "climber"
  type_: PERSON
  salience: 0.301220328
  mentions {
    text {
      content: "climber"
      begin_offset: -1
    }
    type_: COMMON
  }
  mentions {
    text {
      content: "climber"
      begin_offset: -1
    }
    type_: COMMON
  }
  mentions {
    text {
      content: "climber"
      begin_offset: -1
    }
    type_: COMMON
  },
  name: "mountaineer"
  type_: PERSON
  salience: 0.166266277
  mentions {
    text {
      content: "mountaineer"
      begin_offset: -1
    }
    type_: COMMON
  },
  name: "police"
  type_: PERSON
  salience: 0.0261899605
  mentions {
    text {
      content: "police"
      begin_offset: -1
    }
    type_: COMMON
  },
  name: "police"
  type_: PERSON
  salience: 0.0216907505
  mentions {
    text {
      content: "police"
      begin_offset: -1
    }
    type_: COMMON
  },
  name: "climber"
  type_: PERSON
  salience: 0.0152979195
  mentions {
    text {
      content: "climber"
      begin_offset: -1
    }
    type_: 

In [58]:
locations = []
for loc in hashmap['location']:
    locations.append((loc.name, loc.salience))
locations
    

[('Swiss Alps', 0.055160198360681534),
 ('GENEVA', 0.027142273262143135),
 ('British', 0.01848442666232586),
 ('Saas-Fee', 0.012874163687229156),
 ('glacier', 0.007699234876781702),
 ('canton', 0.005676985718309879),
 ('glacier', 0.003711678786203265),
 ('Valais', 0.0033575785346329212),
 ('Saas Valley.It', 0.003118769731372595),
 ('German', 0.0016367161879315972),
 ('Scotland', 0.0010303736198693514)]

In [51]:
hashmap['location'][0].metadata

{'wikipedia_url': 'https://en.wikipedia.org/wiki/Switzerland', 'mid': '/m/06mzp'}

In [60]:
print(hashmap['location'][1].name)
print(hashmap['location'][1].salience)

GENEVA
0.027142273262143135
